> 08 슈팅
- 좌우 키로 내 캐릭터를 이동하고 스페이스 키로 미사일을 발사합니다 .다가오는 외계인을 모두 쏘세요. 바리케이트가 없고 UFO가 출현하지 않는 등 없는것 투성이지만 그래도 한 시대를 풍미했던 게임의 분위기는 엿볼 수 있습니다

In [1]:
#### invader ####
import sys
from random import randint
import pygame
from pygame.locals import QUIT, KEYDOWN, Rect, K_LEFT, K_RIGHT, K_SPACE

pygame.init()
pygame.key.set_repeat(5, 5)
SURFACE = pygame.display.set_mode((600,600))
FPSCLOCK = pygame.time.Clock()

class Drawable: #전체의 그리기 객체의 슈퍼클래스
    pass

class Ship(Drawable): #우주선 클래스
    pass

class Beam(Drawable): #빔 클래스
    pass

class Bomb(Drawable): #폭탄 클래스
    pass

class Alien(Drawable): #외계인 클래스
    pass

def main():

    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            

        SURFACE.fill((0, 0, 0))

        pygame.display.update()
        FPSCLOCK.tick(20)
if __name__ == '__main__':
    main()

c:\Users\고창섭\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\고창섭\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
